In [1]:
import pandas as pd

**Задание 1:** <br>
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [67]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [89]:
movie_fans = (
    ratings
    .groupby('userId', as_index = False)['userId']  # Group By
    .agg({'qty': 'count'}) # Любое название, т.к. у объекта DataFrameGroupBy еще не задано название колонки
    .query('qty > 100') # Having Count(*) > 100
)
# Получены пользователи, выставившие больше 100 оценок
movie_fans.head(5)

,userId,qty
0,1,232
3,4,216
5,6,314
6,7,152
9,10,140


In [96]:
lifetime = (
    movie_fans
    .merge(ratings, on = 'userId', how = 'inner')
    .groupby('userId', as_index = False)
    .agg({'timestamp':[max, min]})
)
# Пользователи, выставившие больше 100 оценок c рассчитанными максимальным и минимальным timestamp пользователя
lifetime.head(5)

userId   timestamp            
                 max         min
0      1   965719662   964980499
1      4  1007574542   945078428
2      6   845556915   845553109
3      7  1176181731  1106635416
4     10  1455619275  1455301553

In [97]:
(lifetime['timestamp']['max'] - lifetime['timestamp']['min']).mean()

44731367.73469388

**Задание 2** <br>
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента <br>

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [99]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [100]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [101]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [145]:
def null_to_zero(result):
    columns_qty = len(result.columns)
    for i in result.columns[columns_qty-3:]:
        result.loc[result[i].isnull(), i] = 0
    return None

In [146]:
result1 = (
    client_base
    .merge(air, how = 'left', on = 'client_id')[['client_id','air_revenue']]
    .merge(rzd, how = 'left', on = 'client_id')
    .merge(auto, how = 'left', on = 'client_id')
)
null_to_zero(result1)
result1

,client_id,air_revenue,rzd_revenue,auto_revenue
0,111,0.0,1093.0,0.0
1,112,0.0,2810.0,0.0
2,113,0.0,10283.0,57483.0
3,114,0.0,5774.0,83.0
4,115,81.0,981.0,912.0
5,116,4.0,0.0,4834.0
6,117,13.0,0.0,98.0
7,118,173.0,0.0,0.0


In [147]:
result2 = (
    client_base
    .merge(air, how = 'left', on = 'client_id')
    .merge(rzd, how = 'left', on = 'client_id')
    .merge(auto, how = 'left', on = 'client_id')
)
null_to_zero(result2)
result2

,client_id,address,air_revenue,rzd_revenue,auto_revenue
0,111,Комсомольская 4,0.0,1093.0,0.0
1,112,Энтузиастов 8а,0.0,2810.0,0.0
2,113,Левобережная 1а,0.0,10283.0,57483.0
3,114,Мира 14,0.0,5774.0,83.0
4,115,ЗЖБИиДК 1,81.0,981.0,912.0
5,116,Строителей 18,4.0,0.0,4834.0
6,117,Панфиловская 33,13.0,0.0,98.0
7,118,Мастеркова 4,173.0,0.0,0.0


**Задание 3** <br>

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают

```
Вариант 1: Выбрать среднюю координату и по ней расчитывать все географические атрибуты (в идеале учитывать и временные интервалы между измерениями местоположения) 
Вариант 2: Отобрать наиболее часто встречающиеся для пользователя гео атрибуты (до заданной точности определять по кординатам: страну, регион, район, город...). Использовать выбранный достоверный атрибут(ы) в качестве сведений о пользователе (например, точно определенный регион)
Вариант 3: Подтягивать информацию с актуальной координаты (последняя по времени измерения)
Варинат 4: Комбинация методов. Выбор метода происходит в зависимости от кол-ва уникальных координат относительно общего числа записей, удаленности координат друг от друга, временного интервала измерений...

```